# DEMO notebook — TEST version


### Set-up 
**Note:** In this notebook, the only class is **`Frame`** (**`Patterns`** and **`Groups`**, used in earlier notebooks, have been removed).

Before anything else, after having checked that the kernel you're running is a Python 3 kernel (it should say Python 3 on the top right, or see Help | About), open up the Git Bash prompt, and do the usual

```
git pull
```

This should pull everythig from GitHub. Then you're ready to run this notebook.

In [1]:
from framenet.data.annotation import get_frame_df, Frame, to_layers, gf_or_target, Groups
from framenet.data.pattern    import *
import pandas as pd, qgrid

## Overview

This notebook contains:

* a set of patterns used in the pattern-matching process
* Displays of valence patterns for a given FrameNet frame, filtered in various ways, and displayed as:
       * tables of valence patterns, with sentence examples
       * San-Key flow diagrams


## Patterns used for pattern-matching process 

We need to do this only once per Notebook. We need those becuase we describe the patterns we look for as Python expressions. The following applies the constructor `Lit` to the strings `'Ext'`, `'Obj'`, `'Dep'`, and `'_'` (our wildcard) and assigns the result to the corresponding Python variables. This way `Ext`, `Obj`, `Dep` and `x` can be combined in the patterns below.


### The symbols that are used in the pattern definitions:

<table class="table table-bordered" style="table-layout: auto !important;">
<tr><th>Symbol</th><th>Meaning</th><th>Category</th></tr>                       
<tr><td>Ext</td><td>external argument (e.g. subject)</td><td>GF</td></tr>
<tr><td>x<td>wildcard element (e.g. verb)</td><td>Target</td></tr>
<tr><td>Obj</td><td>direct object</td><td>GF</td></tr>
<tr><td>Dep</td><td>dependent</td><td>GF</td></tr>
<tr><td>CNI</td><td>Constructional Null Instantiation</td><td>NI</td></tr>
<tr><td>DNI</td><td>Definite Null Instantiation</td><td>NI</td></tr>
<tr><td>INI</td><td>Indefinite Null Instantiation</td><td>NI</td></tr>
</table>



### [*QUESTION*: do we want to add more variables? If so, how?]


### The operators used to create patterns:

<table class="table table-bordered" style="table-layout: auto !important">
<tr><th>The operator</th><th>means</th></tr>                       
<tr><td>&amp;</td><td>_followed by_</td></tr>
<tr><td>`|`</td><td>_one or the other_</td></tr>
<tr><td>`Many1(...)`</td><td>_one or more of ..._</td></tr>
<tr><td>`Many(...)`</td><td>_zero or more of ..._</td></tr>
<tr><td>`Opt(...)`</td><td>_optionally ..._</td></tr>
</table>


In [2]:
Ext, Obj, Dep, x, CNI, DNI, INI = map(Lit, 'Ext Obj Dep _ CNI DNI INI'.split())

### Active transitive, no dependents

In [3]:
p1 = Ext & x & Obj 
p1_NI = Ext & x & Obj & Opt(CNI|DNI|INI)

### Active transitive, plus one or more dependents


In [4]:
p1_dep = Ext & x & Obj & Many1(Dep)
p1_dep_NI = Ext & x & Obj & Many1(Dep) & Opt(CNI|DNI|INI)

### Intransitive or Passive transitive

In [5]:
p2 =  Ext & x 
p2_dep = Ext & x & Many1(Dep)
p2_dep_NI = Ext & x & Many1(Dep) & Opt(CNI|DNI|INI)

### Imperative for 'full' active transitive

This is pattern 'p1_dep_NI', minus the Ext argument

In [6]:
p3 = x & Obj & Many1(Dep) & Opt(CNI|DNI|INI)

### Active Transitive with Dep FE immediately following the verb
* E.g. 
Note that in FN annotation, bare prepositions are given the same PT as full prepositional phrases. 


In [7]:
p4 = Ext & x & Dep & Obj & Many(Dep) & Opt(CNI|DNI|INI)

### Active transitive in relative clauses

In [8]:
p5_rel_subj = Ext & Ext & x & Obj & Many1(Dep) & Opt(CNI|DNI|INI)
p5_rel_obj  = Obj & (Obj) & Ext & x & Many1(Dep) & Opt(CNI|DNI|INI)

## TODO: 
### 1. add sentence examples for the different patterns?
### 2. add relative clause patterns? (if so, add to the table too!

## A table showing the patterns defined in this notebook:

<table>
<tr><th>Desciption</th><th>Pattern</th></tr>                       
<tr><td>active transitive</td><td>Ext > V > Obj </td></tr>
<tr><td>active transitive plus optional NI FEs</td><td>Ext > V > Obj + (NI)</td></tr>
<tr><td>active transitive with dependent(s)</td><td>Ext > V > Obj > Dep</td></tr>
<tr><td>active transitive with dep(s), plus optional NI FEs</td><td>Ext > V > Obj > Dep + (NI)</td></tr>
<tr><td>intransitive/active passive</td><td>Ext > Verb</td></tr>
<tr><td>intransitive/active passive with dep(s)</td><td>Ext > Verb > Dep</td></tr>
<tr><td>intransitive/active passive, plus optional NI FEs</td><td>Ext > Verb > Dep + (NI)</td></tr>
<tr><td>Imperative </td><td>V > Obj > Dep</td></tr>
<tr><td>active trans with v > dep > obj </td><td>Ext > Dep > Obj > Many(Dep) + (NI)</td></tr>
<tr><td>active trans in subj relative clause </td><td>Ext > Ext> Obj > Many(Dep) + (NI)</td></tr>
<tr><td>active trans in 0bj relative clause </td><td>Obj > Obj > Ext> Many(Dep) + (NI)</td></tr>
</table>

        
        

## A table showing the patterns defined in this notebook:

<table>
<tr><th>Name</th><th>Desciption</th><th>Pattern</th><th>Finds</th></tr>                       
<tr><td>p1</td><td>active transitive</td><td>Ext & x & Obj</td><td>Ext > V > Obj </td></tr>
<tr><td>p1_NI<td>active transitive plus optional NI FEs</td><td>Ext & x & Obj & Opt(CNI|DNI|INI)</td><td>Ext > V > Obj + (NI)</td></tr>
<tr><td>p1_dep</td><td>active transitive with dependent(s)</td><td>Ext & x & Obj & Many1(Dep)</td><td>Ext > V > Obj > Dep</td></tr>
<tr><td>p1_dep_NI</td><td>active transitive with dep(s), plus optional NI FEs</td><td>Ext & x & Obj & Many1(Dep) & Opt(CNI|DNI|INI)</td><td>Ext > V > Obj > Dep + (NI)</td></tr>
<tr><td>p2</td><td>intransitive/active passive</td><td>Ext & x </td><td>Ext > Verb</td></tr>
<tr><td>p2_dep</td><td>intransitive/active passive with dep(s)</td><td>Ext & x & Many1(Dep)</td><td>Ext > Verb > Dep</td></tr>
<tr><td>p2_dep_NI</td><td>intransitive/active passive, plus optional NI FEs</td><td>Ext & x & Many1(Dep) & Opt(CNI|DNI|INI)</td><td>Ext > Verb > Dep + (NI)</td></tr>
<tr><td>p3</td><td>Imperative </td><td>x & Obj & Many1(Dep) & Opt(CNI|DNI|INI)</td><td>V > Obj > Dep</td></tr>
<tr><td>p4</td><td>active trans with v > dep > obj </td><td>Ext & x & Dep & Obj & Many1(Dep) & Opt(CNI|DNI|INI)</td><td>Ext > Dep > Obj > Many(Dep) + (NI)</td></tr>
<tr><td>p5_rel_subj</td><td>active trans in subj relative clause </td><td>Ext & Ext & x & Obj & Many1(Dep) & Opt(CNI|DNI|INI)</td><td>Ext > Ext> Obj > Many(Dep) + (NI)</td></tr>
<tr><td>p5_rel_obj</td><td>active trans in 0bj relative clause </td><td>Obj & (Obj) & Ext & x & Many1(Dep) & Opt(CNI|DNI|INI)</td><td>Obj > Obj > Ext> Many(Dep) + (NI)</td></tr>
</table>

        
        

# Select FrameNet `Frame`

`Frame` simply groups together the information related to a specific FN frame. 

In [9]:
cfm = Frame('Cause_fluidic_motion')

### Here are the lexical units for the selected frame:

In [10]:
cfm.groups.data_frame['annotationSet.LU'].unique()

array(['dribble.v', 'spill.v', 'spatter.v', 'splatter.v', 'splash.v',
       'spray.v', 'squirt.v', 'pump.v'], dtype=object)

# Look at different patterns: two methods 

## 1. Display method: show selected patterns as a table

### The `display` method understands the following keywords, all optional:

* `pattern_matcher`: defined as the above expressions, or `None` / unspecified. E.g. pattern_matcher = p1
* `min_count`: the minimum count for sentences matching a pattern; a nonnegative integer number
* `negative`: a boolean specifying whether a pattern should be inverted (when set to 'True', picks up sentences that *do not* match)

The display method also returns a total count of the sentences that meet the display criteria.  
One way to display all of the sentences annotated for a given frame is to use `display`without any parameters: `display()`


### Display example 1:  Look at all of the sentences annotated for the selected frame:

In [11]:
cfm.display(min_count=1)

### Display example 2: Show   'Active Transitive plus dependents' pattern (p1_dep) with more than one example

In [12]:
cfm.display(
    pattern_matcher=(p1_dep), 
    negative=False, 
    min_count=1,
   )

### Display example 4: Show examples after the pattern above (p1_dep) has been filtered out

In [13]:
cfm.display(pattern_matcher = p1_dep, min_count = 0, negative=True)

,$N$,Patterns
1,9,"Ext: Fluid (NP) → v → Dep: Goal (PP) → CNI: Agent (None)` Another scene cut was one where a man 's brains are splattered over a window . ""During the re-entry and completion of BA-1 a large amount of brine was pumped into the well , carrying fines with it and plugging the pores around the well bore .This amplification is produced at the expense of energy pumped into the medium via the population inversion of its atoms or molecules .A third of sewage pumped into the sea is untreated .He felt cold , thick fluid being splashed on him .There were 16 cases of damage to vehicles including £300 damage to a vehicle in Larchfield Street , while paint was sprayed over a vehicle in Rydal Road .When blood tests are taken , we use local anesthesia , sprayed on the skin .The use of systemic fungicides sprayed onto potato crops , and fumigation of stored potatoes , can reduce the incidence of the disease but the spread of infection is greatly facilitated by modern harvesting equipment .A milkman who was temporarily blinded when superglue was squirted into his eyes , has given up his job ."
2,4,"v → Obj: Fluid (NP) → Dep: Goal (PP) → CNI: Agent (None)Fat Trask choked with laughter , spattering half-chewed pasta over another man who slapped him upside the head .Splashing the liquid into a goblet , he carried the drink back to the bed and shoved it brusquely into Isabel 's hand .` When you 've got it full up to the top , splash a bit of Paraffin on the top and put the match to it .Simply spray Hot Shapes onto clean , dry hair before setting to get instant hold with a glossy finish ."
3,3,"Ext: Fluid (NP) → v → Dep: Goal (PP) → CNI: Cause (None)Before the sound of the gunshot , before the Captain 's brains mixed with blood were spattered on to the instrument panel in front of him , Terry thought he heard him say something .And he would be thinking of her , and wanting her , and watching the raindrops as they spattered against the window .His skin was bursting , stretched almost to transparency by the blood pumped down into it ."
4,2,"Ext: Fluid (NP) → v → Obj: Goal (NP) → Dep: Result (VPing) → INI: Agent (None)More bullets spattered the front of the cottage , drilling lines back and forth in the stonework .A roar greeted his effort at authority and dirt spattered him , making him blink and cower ."
5,2,"Dep: Time (AVP) → Ext: Fluid (NP) → v → Obj: Goal (NP) → INI: Cause (None)This time a few , reluctant drops spattered the millpond surface but did not disturb it .Then the monstrous green body exploded like a balloon of filth , spattering the walls of the cavern and the cages of the cowering young prisoners -- the last time he/she would set a mark upon them ."
6,2,"Obj: Fluid (NP) → Obj: Fluid (NP) → Ext: Agent (NP) → v → Dep: Goal (PP) → Dep: Time (PP)Do n't forget to add the Aquasafe at this stage , to remove the chlorines and chloramines that our water boards are pumping into the tapwater at the moment .It was part of the defoliant Agent Orange , that the US forces sprayed on Vietnamese forests during the 1960s and 1970s ."
7,2,"Ext: Fluid (NP) → v → Dep: Goal (PP) → Dep: Agent (PP)A man who had superglue squirted in his face by muggers may have lost the sight of one eye .Well , he supposedly had drink spilled on him by a coachman so , by way of revenge , he poured brandy over the man and ignited him ."
8,2,"v → Obj: Fluid (NP) → Dep: Goal (PP) → DNI: Agent (None)It was Barney who returned to it first , dribbling cream into his coffee and watching it swirl into a spiral on the surface .Beyond that , you can usually buy extensions covering accidental damage to the buildings , eg ; by putting your foot through the bedroom ceiling , or to the contents , eg , by spilling paint on a carpet ."
9,2,"Ext: Agent (NP) → v → Obj: Fluid (NP) → INI: Area (None)My hand jerked so that I almost spilled my wine .Jenna jumped up , almost spilling her coffee , making a hasty grab for the delicate cup and amusing him even more ."
10,1,Ext: Flui

## 2. Flow diagram method: show selected patterns as a Sankey flow diagram

### The `select` method understands some of the same keywords as the  `display` method (above):

* `pattern_matcher`: defined as the above expressions, or `None` / unspecified. E.g. pattern_matcher = p1
* `negative`: a boolean specifying whether a pattern should be inverted (when set to 'True', picks up sentences that *do not* match)

The select method also returns a total count of the sentences that meet the display criteria.  

### It is also possible to specify whether 'noncore' FEs (frame elements) should be included in the flow diagram: 

* `diagram` method takes an optional keyword argument to select noncore `FE`s. The default value is `True`.
* the `diagram` method also takes an optional argument to specify diagram size. E.g. size=(960, 768)


### Note: this method ignore phrase type (PT) distinctions in the annotated data. 
          E.g. it groups together Dep: Path" regardless of whether the phrase type is PP or AVP.


### Flow diagram method example 1: Chart for the Active Transitive plus dependents' pattern (p1_dep), fo core FEs only

In [14]:
cfm.select(p1_dep, negative=False).diagram(noncore=False, size=(640, 320))

Written 13 records.


<IPython.core.display.Javascript object>

In [22]:
cfm.select(p2_dep_NI, negative=False).diagram(noncore=False, size=(640, 320))

Written 16 records.


<IPython.core.display.Javascript object>

## More examples we worked on

### The `Filling` frame

Comment here.

In [16]:
filling = Frame('Filling')
filling.display(pattern_matcher=p1_dep, min_count=0, negative=False)

In [17]:
filling.display()

In [18]:
filling.select(p1_dep).diagram(noncore=True)

Written 15 records.


<IPython.core.display.Javascript object>

In [25]:
cfm.display(pattern_matcher=p4)

,$N$,Patterns
1,1,"Ext: Fluid (NP) → v → Dep: Manner (PP) → Obj: Area (NP) → INI: Cause (None)The rain , which had been coming and going , began to pour down again , spattering in big gobs all over the glass around them ."
2,1,Ext: Cause (NP) → v → Dep: Path (AVP) → Obj: Fluid (NP)This in turn helps your body to pump out toxins and excess fluid .
3,1,"Ext: Agent (NP) → v → Dep: Goal (AVP) → Obj: Fluid (NP) → Dep: Time (PP)Surprisingly , it 's normal for some women to squirt out a fluid at the moment of orgasm ."


In [26]:
cfm.display(pattern_matcher=p1_dep_NI)

In [27]:
cfm.select(p1_dep_NI).diagram(noncore=False, size=(640, 320))

Written 14 records.


<IPython.core.display.Javascript object>

In [28]:
cfm.select(p2_dep_NI).diagram(noncore=False)

Written 16 records.


<IPython.core.display.Javascript object>

In [29]:
cfm.select(p1_dep_NI).diagram(noncore=False, size=(640, 320))

Written 14 records.


<IPython.core.display.Javascript object>

In [ ]:
cm = Frame('Cause_motion')
cm.display(
    pattern_matcher=(), 
    negative=False, 
    min_count=0,
)

In [36]:
cm.display(
    pattern_matcher=(p3), 
    negative=False, 
    min_count=0,
)

,$N$,Patterns
1,11,"v → Obj: Theme (NP) → Dep: Goal (PP) → CNI: Agent (None)Odd-Knut makes coffee using the method we have become familiar with -- chuck the coffee in a pot , add snow to taste , boil on the stove , when empty add coffee and/or snow as necessary .What better way to test the mettle of the new Calibra 16v than to pitch it directly against the class best , VW 's Corrado G60 ?Press each block firmly in place , and weight them with something heavy until the adhesive has set .CAPILLARY REFILL TEST : This involves pressing your thumb onto the horse 's gum .Again , change hands , push the tiller to where you were sitting , watch for the boom , as it swings across , straighten up and sit down on the new side .A TIP : If a power failure occurs ( or the mains have to be turned off ) temperature loss can be much reduced by just throwing a blanket from the bed over the aquarium .Be very careful of old fluorescent light tubes , which can explode if dropped , and never puncture old aerosol cans or throw them on a fire .Do not throw rubbish onto an open fire in the living room .Throws her bouquet to her unmarried friends before she leaves .Toss the potatoes in the butter , spoon into a warmed serving dish and garnish with more sprigs of basil .Or moving it to Lubbock or somewhere possibly is not the answer ."
2,6,"v → Obj: Theme (NP) → Dep: Path (PP) → CNI: Agent (None)I ca n't afford to keep chucking two quid down the drain .The tank blew up , hurling the Vauxhall through the closed garage doors , which caused twenty more gallons of petrol to explode , blowing the front off the house .And if your sauce has turned lumpy , press it through a sieve .` Shove it up your armpit ! ""If people want to escape urban gloom I have a good suggestion : throw your TV , video and stereo through the window , lock your car in the garage and buy a bicycle .That bit about throwing the stone up the alley to make him rush off after it , I got that from a detective story ."
3,5,"v → Obj: Theme (NP) → Dep: Source (PP) → CNI: Agent (None)I was impressed by the streamlined efficiency of the Surgicentre ( if a little dismayed by the check 'em in , chop it off , chuck 'em out attitude ) .Take him away and chuck him off the derrick . ""Do not fling me from your houseThe aim is to leap around in an inflated suit and push your opponent off the mat .Alternatively , hold a heavy medicine ball in both hands , bend your elbows and thrust it from you ."
4,3,"v → Obj: Goal (NP) → Dep: Theme (NP) → CNI: Agent (None)` Chuck me a can ! ""` Chuck us a hammer , Cyril , "" he shouted down bossily , ` the brickwork 's not too bad .Shove us the marmalade , Jo . """
5,2,"v → Obj: Theme (NP) → Dep: Goal (AVP) → CNI: Agent (None)Cameras won , and in chucking my fishing bag ashore , the 4lb breaking-strain cast snapped and the trout was gone .In other words , fire up the Transit , shove every member of the band inside , come down and make thorough , absolute pigs of yourselves ."
6,2,v → Obj: Theme (NP) → Dep: Source (PP) → Dep: Goal (PP) → CNI: Agent (None)Push empty needles from non working to working position and make a hem by placing loops of first row worked in main yarn evenly along the row .This article inaugurated Rothermere 's massive press campaign which helped to thrust the BUF from comparative obscurity into the limelight .
7,1,v → Obj: Theme (NP) → Dep: Path (PP) → Dep: Distance (PP) → CNI: Agent (None)Cut a small hole at the bottom point and push the green covered pencil or stick through the hole for about 2cm ( Fig 3 ) .
8,1,"v → Obj: Theme (NP) → Dep: Path (AVP) → CNI: Agent (None)Knee : Protect your groin and throat , yell , and thrust your hip forward for extra power ."
9,1,v → Obj: Theme (NP) → Dep: Result (AVP) → Dep: Manner (AVP) → CNI: Agent (None)Add to bowl and toss salad ingredients together gently .
10,1,"v → Obj: Theme (NP) → Dep: Path (PP) → Dep: Path (PP) → CNI: Agent (None)To learn to juggle , take one ball and practise tossing it from hand to 

In [35]:
# NOTE: THIS FAILS because there are 2 different annoSets (6547982, 6547992) that disagree. 
# We (wrongly, apparently) assume that FE annotations are unique.

fm = Frame('Fluidic_motion')
fm.display(
    pattern_matcher=(p1_dep_NI), 
    negative=False, 
    min_count=2,
)

'Problem with group'
[[{'annotationSet.ID': 6547982,
   'annotationSet.LU': 'spill.v',
   'annotationSet.status': 'MANUAL',
   'label.coreFE': nan,
   'label.end': 0.0,
   'label.feID': nan,
   'label.itype': nan,
   'label.name': 'Metaphor',
   'label.start': 0.0,
   'layer.name': 'Sent',
   'layer.rank': 1,
   'sentence.ID': 4102337,
   'sentence.aPos': 0,
   'sentence.corpID': 205.0,
   'sentence.docID': 23733.0,
   'sentence.paragNo': 1.0,
   'sentence.sentNo': 1,
   'text.contents': 'Paula_Zahn : Questions about the facts or what were '
                    'presented as facts that led the United States into the '
                    'war in Iraq spilled into open warfare today on the Senate '
                    'floor .'},
  {'annotationSet.ID': 6547992,
   'annotationSet.LU': 'spill.v',
   'annotationSet.status': 'MANUAL',
   'label.coreFE': nan,
   'label.end': 0.0,
   'label.feID': nan,
   'label.itype': nan,
   'label.name': 'Metaphor',
   'label.start': 0.0,
   'layer.name':

   'label.start': 145.0,
   'layer.name': 'FE',
   'layer.rank': 1,
   'sentence.ID': 4102337,
   'sentence.aPos': 0,
   'sentence.corpID': 205.0,
   'sentence.docID': 23733.0,
   'sentence.paragNo': 1.0,
   'sentence.sentNo': 1,
   'text.contents': 'Paula_Zahn : Questions about the facts or what were '
                    'presented as facts that led the United States into the '
                    'war in Iraq spilled into open warfare today on the Senate '
                    'floor .'},
  {'annotationSet.ID': 6547982,
   'annotationSet.LU': 'spill.v',
   'annotationSet.status': 'MANUAL',
   'label.coreFE': nan,
   'label.end': 149.0,
   'label.feID': nan,
   'label.itype': nan,
   'label.name': 'Dep',
   'label.start': 145.0,
   'layer.name': 'GF',
   'layer.rank': 1,
   'sentence.ID': 4102337,
   'sentence.aPos': 0,
   'sentence.corpID': 205.0,
   'sentence.docID': 23733.0,
   'sentence.paragNo': 1.0,
   'sentence.sentNo': 1,
   'text.contents': 'Paula_Zahn : Questions about the f

ValueError: Item (FE -> Configuration) already in dictionary {'FE': 'Goal', 'core': True, 'GF': 'Dep', 'PT': 'PP'} ([('FE', 'Goal'), ('core', True), ('GF', 'Dep'), ('PT', 'PP'), ('FE', 'Configuration'), ('GF', 'Dep'), ('PT', 'PP')])

In [ ]:
cm = Frame('Cause_motion')
cm.display(
    pattern_matcher=(p1_dep_NI) | (p2_dep_NI) | (p3) | (p4) | (p5_rel_subj) | (p5_rel_obj), 
    negative=True, 
    min_count=2,
)

In [ ]:
cm = Frame('Cause_motion')
cm.display(
    pattern_matcher=(p1_dep_NI) | (p2_dep_NI) | (p3) | (p4) | (p5_rel_subj) | (p5_rel_obj), 
    negative=True, 
    min_count=2,
)

In [ ]:
    cm.display(pattern_matcher=(p5_rel_obj))

In [29]:
cm.select(p1_dep_NI).diagram(noncore=False)

Written 22 records.


<IPython.core.display.Javascript object>

In [33]:
cm.select(p2_dep_NI).diagram(noncore=False, size=(640, 480))

Written 39 records.


<IPython.core.display.Javascript object>

In [ ]:
cm.display(
    pattern_matcher= None,
    negative=False, 
    min_count=8,
    include_sentences=False
)

In [ ]:
cm.display(pattern_matcher=p5_rel_obj, min_count=0)

In [ ]:
cm.display(pattern_matcher=p4, min_count=0)

In [ ]:
cm.select(p4).diagram(size=(960, 768))

The end.